In [1]:
import Quandl
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation
import datetime
from pylab import *
from sklearn import metrics
#%matplotlib inline
%matplotlib qt

In [2]:
df = Quandl.get("WIKI/GOOGL")
print(df)
pickle_out = open('GOOGL.pickle','wb')
pickle.dump(df, pickle_out)
pickle_out.close() 

              Open      High       Low   Close      Volume  Ex-Dividend  \
Date                                                                      
2004-08-19  100.00  104.0600   95.9600  100.34  44659000.0          0.0   
2004-08-20  101.01  109.0800  100.5000  108.31  22834300.0          0.0   
2004-08-23  110.75  113.4800  109.0500  109.40  18256100.0          0.0   
2004-08-24  111.24  111.6000  103.5700  104.87  15247300.0          0.0   
2004-08-25  104.96  108.0000  103.8800  106.00   9188600.0          0.0   
2004-08-26  104.95  107.9500  104.6600  107.91   7094800.0          0.0   
2004-08-27  108.10  108.6200  105.6900  106.15   6211700.0          0.0   
2004-08-30  105.28  105.4900  102.0100  102.01   5196700.0          0.0   
2004-08-31  102.30  103.7100  102.1600  102.37   4917800.0          0.0   
2004-09-01  102.70  102.9700   99.6700  100.25   9138200.0          0.0   
2004-09-02   99.19  102.3700   98.9400  101.51  15118600.0          0.0   
2004-09-03  100.95  101.7

In [11]:
GOOGL_data_complete = pd.read_pickle('GOOGL.pickle')
GOOGL_data_complete = GOOGL_data_complete[['Adj. Open',  'Adj. High',  'Adj. Low',  'Adj. Close', 'Adj. Volume']]
GOOGL_data_complete['HL_PCT'] = (GOOGL_data_complete['Adj. High'] - GOOGL_data_complete['Adj. Low']) / GOOGL_data_complete['Adj. Close'] * 100.0
GOOGL_data_complete['PCT_change'] = (GOOGL_data_complete['Adj. Close'] - GOOGL_data_complete['Adj. Open']) / GOOGL_data_complete['Adj. Open'] * 100.0
GOOGL_data_complete = GOOGL_data_complete[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]
GOOGL_data_complete['Forecast'] = GOOGL_data_complete['Adj. Close'].shift(-1)
#plt.plot(GOOGL_data_complete['PCT_change'])
#plt.xlabel('Date')
#plt.ylabel('PCT_change')
#print(GOOGL_data_complete.head())

print(GOOGL_data_complete.tail())

            Adj. Close    HL_PCT  PCT_change  Adj. Volume  Forecast
Date                                                               
2016-05-17      720.19  2.478513   -1.486882    1631175.0    721.78
2016-05-18      721.78  1.461664    0.456507    1607604.0    715.31
2016-05-19      715.31  1.425256   -0.416261    1488030.0    721.71
2016-05-20      721.71  1.758324    0.732769    1710030.0    717.24
2016-05-23      717.24  0.914603   -0.380566    1228165.0       NaN


In [12]:
GOOGL_data = GOOGL_data_complete[:len(GOOGL_data_complete)-30]
print(GOOGL_data.head())
print(GOOGL_data.tail())

            Adj. Close    HL_PCT  PCT_change  Adj. Volume  Forecast
Date                                                               
2004-08-19      50.170  8.072553    0.340000   44659000.0    54.155
2004-08-20      54.155  7.921706    7.227007   22834300.0    54.700
2004-08-23      54.700  4.049360   -1.218962   18256100.0    52.435
2004-08-24      52.435  7.657099   -5.726357   15247300.0    53.000
2004-08-25      53.000  3.886792    0.990854    9188600.0    53.955
            Adj. Close    HL_PCT  PCT_change  Adj. Volume  Forecast
Date                                                               
2016-04-05      758.57  0.958382    0.058038    1210331.0    768.07
2016-04-06      768.07  1.577981    1.349889    1250254.0    760.12
2016-04-07      760.12  1.560280   -0.679454    1221521.0    759.47
2016-04-08      759.47  1.495780   -0.835651    1158783.0    757.54
2016-04-11      757.54  1.304222   -1.033379    1576119.0    764.32


In [13]:
GOOGL_train = GOOGL_data[:(int(len(GOOGL_data)) - int(len(GOOGL_data)*0.25))]
X = GOOGL_train[['Adj. Close', 'HL_PCT','PCT_change','Adj. Volume']]
y = GOOGL_train[['Forecast']]

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)
print(X_train.tail())
print(y_train.tail())

            Adj. Close     HL_PCT  PCT_change  Adj. Volume
Date                                                      
2010-05-05     254.880   2.991604    1.752565    9155200.0
2010-12-01     282.175   1.624878    0.239787    7500600.0
2006-02-28     181.310  16.278749   -7.777213   78796300.0
2010-05-12     252.695   1.986585   -1.298727    7695800.0
2012-08-22     338.590   2.052630    1.468429    3814500.0
            Forecast
Date                
2010-05-05   249.335
2010-12-01   285.910
2006-02-28   182.400
2010-05-12   255.440
2012-08-22   338.400


In [14]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
confidence = linreg.score(X_test, y_test)
print(confidence)


0.995747061071


In [15]:
print(X_test.head())
print(X_test.tail())

            Adj. Close    HL_PCT  PCT_change  Adj. Volume
Date                                                     
2012-03-07     303.400  0.878378   -0.369428    2526400.0
2004-10-13      70.450  2.462740   -1.688529   19766200.0
2013-04-08     387.425  1.438988   -0.500803    5658500.0
2012-02-13     306.100  0.623979    0.278460    3628900.0
2011-09-26     265.945  3.700013    0.880038    5263100.0
            Adj. Close    HL_PCT  PCT_change  Adj. Volume
Date                                                     
2005-05-05     113.490  1.207155   -0.717348   15004100.0
2005-09-07     147.435  3.465934    3.141068   14984000.0
2012-09-04     340.520  1.688594   -0.512746    3775400.0
2004-12-15      89.890  2.241629    1.005674   22919000.0
2005-10-24     174.325  2.039294    1.537700   18844500.0


In [16]:
GOOGL_test_forecast = GOOGL_data_complete[len(GOOGL_data_complete)-30:]
#print(GOOGL_test_forecast)
actualClose = GOOGL_test_forecast['Adj. Close']
#print(actualClose)
#print(GOOGL_test_forecast)
GOOGL_test_forecast = np.array(GOOGL_test_forecast.drop(['Forecast'], 1))
#print(GOOGL_test_forecast)
y_pred = linreg.predict(GOOGL_test_forecast)
#print(metrics.mean_absolute_error(true, pred))
#print(y_pred)
index = actualClose.index
#index = index.drop(index[0])
#print(index)
#print(len(y_pred))
    
prediction = pd.DataFrame(y_pred, index)
print(prediction)

#print(len(prediction))
prediction[0] = prediction[0].shift(1)
print(prediction)
#print(actualClose)

plt.plot(prediction, color='blue')
plt.plot(actualClose, color='red')
plt.legend(loc=1)
plt.xlabel('Date')
plt.ylabel('Price')
plt.xticks(rotation=45)

plt.show()


                     0
Date                  
2016-04-12  763.058468
2016-04-13  770.668856
2016-04-14  774.212797
2016-04-15  778.809048
2016-04-18  786.391180
2016-04-19  774.881109
2016-04-20  773.721318
2016-04-21  778.751650
2016-04-22  736.092397
2016-04-25  741.059727
2016-04-26  724.061217
2016-04-27  720.250627
2016-04-28  703.784312
2016-04-29  706.848902
2016-05-02  713.351396
2016-05-03  707.407810
2016-05-04  710.284097
2016-05-05  713.670581
2016-05-06  723.995719
2016-05-09  728.005269
2016-05-10  738.262383
2016-05-11  729.408700
2016-05-12  726.953482
2016-05-13  723.771048
2016-05-16  729.135842
2016-05-17  718.997876
2016-05-18  720.676608
2016-05-19  714.239873
2016-05-20  720.569300
2016-05-23  716.222603
                     0
Date                  
2016-04-12         NaN
2016-04-13  763.058468
2016-04-14  770.668856
2016-04-15  774.212797
2016-04-18  778.809048
2016-04-19  786.391180
2016-04-20  774.881109
2016-04-21  773.721318
2016-04-22  778.751650
2016-04-25 

In [17]:
#print(GOOGL_test_y)
print(y_pred)
#actual = GOOGL_test_y[['Forecast']]

#print(actual)
#compare = pd.concat([pd.DataFrame(y_pred), GOOGL_test_y])
#print(compare)
#compare = GOOGL_data_complete['Forecast'] = GOOGL_data_complete['Adj. Close'].shift(-1)
#plt.plot(y_pred)
#plt.plot(actual)

#plt.xlabel('Date')
#plt.ylabel('Price')

[[          nan]
 [ 763.05846848]
 [ 770.6688564 ]
 [ 774.21279665]
 [ 778.80904789]
 [ 786.39118043]
 [ 774.88110925]
 [ 773.72131768]
 [ 778.75165028]
 [ 736.09239713]
 [ 741.05972704]
 [ 724.06121724]
 [ 720.25062719]
 [ 703.78431245]
 [ 706.84890213]
 [ 713.35139624]
 [ 707.40781003]
 [ 710.28409697]
 [ 713.67058091]
 [ 723.99571926]
 [ 728.0052689 ]
 [ 738.26238311]
 [ 729.40869979]
 [ 726.9534822 ]
 [ 723.77104845]
 [ 729.13584218]
 [ 718.99787646]
 [ 720.67660836]
 [ 714.23987284]
 [ 720.56929986]]


In [18]:
GOOGL_test_forecast = GOOGL_data_complete[len(GOOGL_data_complete)-30:]
#print(GOOGL_test_forecast.head())
#print(GOOGL_test_forecast.tail())